# Updating GridStatus data
Gridstatus data is archived weekly by an action in the [`deployment-gap-model-archiver` repo](https://github.com/deployment-gap-model-education-fund/deployment-gap-model-archiver/actions). The files are archived in a GCS bucket with [object versioning enabled](https://cloud.google.com/storage/docs/object-versioning). A generation IDs is created for each version of the archived parquet file. To update the gridstatus data, grab the generation id for each parquet for desired date and update the `ISO_QUEUE_VERSIONS` dictionary in `dbcp.extract.gridstatus_isoqueues`. Use this notebook to validate and compare the dates of the most recent projects for each ISO.

In [34]:
from dbcp.extract.gridstatus_isoqueues import extract, ISO_QUEUE_VERSIONS
import pandas as pd


old_dfs = extract(ISO_QUEUE_VERSIONS)

In [35]:
ISO_QUEUE_VERSIONS: dict[str, str] = {
    "miso": "1719774997006069",
    "miso-pre-2017": "1709776311574737",
    "caiso": "1719774997530790",
    "pjm": "1719774998059470",
    "ercot": "1719774998544416",
    "spp": "1719774998998901",
    "nyiso": "1719774999497797",
    "isone": "1719774999940225",
}

new_dfs = extract(ISO_QUEUE_VERSIONS)

In [36]:
comp = []
for iso in ISO_QUEUE_VERSIONS:
    # df["Queue Date"] = pd.to_datetime(df["Queue Date"])
    new_df = new_dfs[iso]
    new_df["Queue Date"] = pd.to_datetime(new_df["Queue Date"])
    
    old_df = old_dfs[iso]
    old_df["Queue Date"] = pd.to_datetime(old_df["Queue Date"])
    
    comp.append((old_df["Queue Date"].max(), new_df["Queue Date"].max(), iso))

In [37]:
comp = pd.DataFrame(comp, columns=["old_max_date", "new_max_date", "iso"])

In [38]:
comp["max_date_diff"] = comp.new_max_date - comp.old_max_date

In [39]:
comp

,old_max_date,new_max_date,iso,max_date_diff
0,2024-03-15 04:00:00+00:00,2024-06-07 04:00:00+00:00,miso,84 days
1,2023-04-03 04:00:00+00:00,2023-04-03 04:00:00+00:00,miso-pre-2017,0 days
2,2023-03-02 08:00:00,2023-03-02 08:00:00,caiso,0 days
3,2023-07-08 00:00:00,2023-07-08 00:00:00,pjm,0 days
4,2024-03-29 00:00:00,2024-05-20 00:00:00,ercot,52 days
5,2024-02-07 00:00:00,2024-05-07 00:00:00,spp,90 days
6,2024-03-30 00:00:00,2024-05-23 00:00:00,nyiso,54 days
7,2024-04-03 00:00:00,2024-06-26 00:00:00,isone,84 days
